In [1]:
import torch
from byol_pytorch.byol import BYOL
from byol_pytorch.NN import Encoder
from torchvision import models

In [2]:
import sys
sys.path.append('../pytorch-scarf')
from cancerclassification.data import *
from cancerclassification.NN import *

# Load Data

In [ ]:
#x_unlabel = read_process_data_TCGA_unlabel('../nn_baseline/data/pretrain_data.parquet')

In [3]:
x_unlabel = np.random.rand(100, 56902)
print(x_unlabel.shape)

(100, 56902)


In [4]:
y_unlabel = np.random.rand(100, 1)
print(y_unlabel.shape)

(100, 1)


In [5]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(device)

cuda


# Define MLP

In [6]:
nb_classes = len(np.unique(y_unlabel))
nb_classes

100

In [7]:
config_nn = {
            "epochs":100,
            "lr_init":4e-4,
            "early_stop":5,
            "optim":optim.Adam,
            "bn":True,
            "dropout_rate":0.0
        }

nb_classes = len(np.unique(y_unlabel))

encoder = Encoder(input_dim=x_unlabel.shape[1], output_dim=64, dropout_rate=0.0).to(device)

In [8]:
learner = BYOL(
    encoder,
    num_features = x_unlabel.shape[1],
    hidden_layer = 2
)

torch.Size([4, 56902])
call get_target_encoder


/shared/ifbstor1/projects/ssrl_tabular/BYOL/byol_pytorch/NN.py:54: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  x = F.softmax(x)


In [9]:
x_unlabel.shape

(100, 56902)

In [10]:
BYOL

byol_pytorch.byol.BYOL

In [11]:
dataset = CancerDatasetTCGA(x_unlabel, y_unlabel, device=device)
dataloader = torch.utils.data.DataLoader(dataset, batch_size = 32, shuffle=False)

In [13]:
opt = torch.optim.Adam(learner.parameters(), lr=1e-4)

for _ in range(10):
    print(_)
    print(next(learner.online_encoder.parameters()))
    for batch in dataloader :
        x, _ = batch
        loss = learner(x)
        loss.requires_grad_()
        opt.zero_grad()
        loss.backward()
        opt.step()
        #print(next(learner.target_encoder.parameters()))
        print(loss)
        learner.update_moving_average()
    print(next(learner.online_encoder.parameters()))

0
Parameter containing:
tensor([[ 0.0018,  0.0036,  0.0010,  ...,  0.0036,  0.0032,  0.0030],
        [-0.0022, -0.0009,  0.0015,  ...,  0.0027, -0.0036, -0.0039],
        [ 0.0036,  0.0008,  0.0033,  ..., -0.0033, -0.0026,  0.0028],
        ...,
        [-0.0039,  0.0034, -0.0015,  ...,  0.0004, -0.0008,  0.0039],
        [ 0.0008,  0.0027,  0.0015,  ...,  0.0011, -0.0041, -0.0009],
        [ 0.0008,  0.0003, -0.0037,  ...,  0.0002,  0.0040,  0.0034]],
       device='cuda:0', requires_grad=True)
torch.Size([64, 56902])
tensor(4.0271, device='cuda:0', requires_grad=True)
torch.Size([64, 56902])
tensor(4.0490, device='cuda:0', requires_grad=True)
torch.Size([64, 56902])
tensor(4.0357, device='cuda:0', requires_grad=True)
torch.Size([8, 56902])
tensor(3.9698, device='cuda:0', requires_grad=True)
Parameter containing:
tensor([[ 0.0018,  0.0036,  0.0010,  ...,  0.0036,  0.0032,  0.0030],
        [-0.0022, -0.0009,  0.0015,  ...,  0.0027, -0.0036, -0.0039],
        [ 0.0036,  0.0008,  0.003

KeyboardInterrupt: 

In [15]:
print(next(encoder.parameters()))

Parameter containing:
tensor([[ 9.2365e-05, -1.1765e-03,  2.1687e-03,  ...,  1.6639e-03,
         -3.8646e-03,  4.1915e-03],
        [-1.7250e-03, -1.0296e-03,  3.6149e-03,  ..., -2.6250e-03,
          3.3400e-03, -1.2343e-03],
        [ 6.4060e-04,  3.5540e-03,  4.0484e-03,  ...,  1.8135e-03,
         -1.8143e-03, -9.8861e-04],
        ...,
        [-3.9542e-03, -2.8534e-03,  3.7906e-03,  ..., -2.4569e-04,
         -1.0572e-03,  1.0169e-03],
        [ 3.2922e-03,  3.9904e-03,  3.3629e-03,  ...,  2.2040e-03,
         -2.4401e-03,  2.6454e-03],
        [ 4.1817e-03, -8.3781e-04,  2.0930e-03,  ...,  3.2132e-03,
          3.9236e-03,  3.5589e-03]], device='cuda:0', requires_grad=True)
